In [1]:
import torch
import numpy as np

In [38]:
#Tutorial 1: tensors
data = [[1, 2], [3, 4]]
x_data = torch.tensor(data)
print(x_data)
np_array = np.array(data)
x_np = torch.from_numpy(np_array)
print(x_np)
x_ones = torch.ones_like(x_data)
print(x_ones)
x_rand = torch.rand_like(x_data, dtype=torch.float)
print(x_rand)
shape = (2, 3,)
rand_tensor = torch.rand(shape)
print(rand_tensor)
ones_tensor = torch.ones(shape)
print(ones_tensor)
zeros_tensor = torch.zeros(shape)
print(zeros_tensor)
tensor = torch.rand(3,4)
print(tensor)
print(tensor.shape)
print(tensor.dtype)
print(tensor.device)

if torch.cuda.is_available():
    print('Cuda is available')
    tensor = tensor.to('cuda')
    print(tensor.device)
    
tensor = torch.ones(4,4)
tensor[:,1] = 0
print(tensor)
t1 = torch.cat([tensor, tensor, tensor], dim=0)
print(t1)
print(tensor.mul(tensor))
print(tensor*tensor)
print(tensor.matmul(tensor.T))
print(tensor @ tensor.T)
print(tensor)
tensor.add_(5)
print(tensor)
t = torch.ones(5)
print(t)
n = t.numpy()
print(n)
t.add_(1)
print(t)
print(n)
n = np.ones(5)
t = torch.from_numpy(n)
print(n)
print(t)
np.add(n, 1, out=n)
print(n)
print(t)

tensor([[1, 2],
        [3, 4]])
tensor([[1, 2],
        [3, 4]])
tensor([[1, 1],
        [1, 1]])
tensor([[0.3479, 0.4615],
        [0.6690, 0.9491]])
tensor([[0.7225, 0.7713, 0.7163],
        [0.3693, 0.1485, 0.1404]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[0.6863, 0.5486, 0.5915, 0.8688],
        [0.7971, 0.8172, 0.0668, 0.3894],
        [0.8990, 0.3120, 0.0658, 0.6671]])
torch.Size([3, 4])
torch.float32
cpu
Cuda is available
cuda:0
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
    

In [39]:
#Tutorial 2: autograd
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1,3,64,64)
labels = torch.rand(1,1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ryley/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

100.0%


In [42]:
prediction = model(data)

In [43]:
loss = (prediction - labels).sum()
loss.backward()

In [44]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [45]:
optim.step()

In [52]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)
print(a.grad)
print(9*a**2 == a.grad)
print(b.grad)
print(-2*b == b.grad)

tensor([36., 81.])
tensor([True, True])
tensor([-12.,  -8.])
tensor([True, True])


In [74]:
#Tutorial 3: nn
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # 5*5 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [75]:
params = list(net.parameters())
print(len(params))
print(params[1].size())
print(params[1])

10
torch.Size([6])
Parameter containing:
tensor([-0.0163,  0.0574,  0.0554, -0.1532,  0.1223,  0.0481],
       requires_grad=True)


In [84]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

net.zero_grad()
out.backward(torch.randn(1,10))

tensor([[-0.0102, -0.0265,  0.0292,  0.0339,  0.0700,  0.0485,  0.0878, -0.0930,
          0.0049,  0.0948]], grad_fn=<AddmmBackward0>)


In [94]:
output = net(input)
target = torch.randn(10)
print(target)
target = target.view(1,-1)
print(target)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

tensor([-0.9270, -1.3390,  0.4967,  0.5881, -0.1596,  0.6156, -1.1772,  1.8254,
         1.3489,  0.5544])
tensor([[-0.9270, -1.3390,  0.4967,  0.5881, -0.1596,  0.6156, -1.1772,  1.8254,
          1.3489,  0.5544]])
tensor(1.0762, grad_fn=<MseLossBackward0>)


In [95]:
net.zero_grad()

print('before backward')
print(net.conv1.bias.grad)

loss.backward()

print('after backward')
print(net.conv1.bias.grad)

before backward
None
after backward
tensor([ 0.0041, -0.0021, -0.0152, -0.0029, -0.0041, -0.0024])


In [98]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [99]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad()
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()